In [ ]:
pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 26.2 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF

def pdf_to_text(pdf_path):
    text = ""
    pdf_document = fitz.open(pdf_path)

    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text += page.get_text()

    pdf_document.close()
    return text

# Replace "your_pdf_file.pdf" with the actual path to your PDF file
pdf_file = "test-2.pdf"
extracted_text = pdf_to_text(pdf_file)

output_text_file = "test-2.txt"

# Write the extracted text to the output text file
with open(output_text_file, 'w', encoding='utf-8') as file:
    file.write(extracted_text)

print(f"Extracted text saved to {output_text_file}")

Extracted text saved to test-2.txt


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import joblib

data = {
    "text": [
        "Legal action may be taken in case of disputes.",
        "The Government may terminate this lease in whole or in part at any time.",
        "Any improper conduct will result in termination.",
        "Renewal options available.",
        "Recapture clause is included.",
        "Tax implications discussed.",
        "continuing for a period of 10 years, with a firm term of five (5) years subject to termination and renewal",
        "This lease is subject to real estate tax adjustment as established in Paragraph",
        "fully recaptured",
        "writing to the Lessor the Government may terminate this lease in whole or part and no rental shall "

        # Add more examples here...
    ],
    "insight": [
        "legal_action",
        "terminate",
        "improper_conduct",
        "renewal",
        "recapture",
        "tax",
        "renewal",
        "tax_adjustment",
        "recaptured",
        "terminate_this_lease"
        # Add corresponding insights for each example...
    ]
}

df = pd.DataFrame(data)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

vectorizer = CountVectorizer(lowercase=True)
X_train = vectorizer.fit_transform(train_df['text'])
X_test = vectorizer.transform(test_df['text'])

classifier = MultinomialNB()
classifier.fit(X_train, train_df['insight'])

trained_model_filename = "trained_model.joblib"
joblib.dump(classifier, trained_model_filename)

vectorizer_filename = "vectorizer.joblib"
joblib.dump(vectorizer, vectorizer_filename)

predictions = classifier.predict(X_test)

print(classification_report(test_df['insight'], predictions))

new_lease_text = "Legal proceedings may arise in case of violations."
new_lease_features = vectorizer.transform([new_lease_text])
predicted_insight = classifier.predict(new_lease_features)
print("Predicted Insight:", predicted_insight[0])


                      precision    recall  f1-score   support

          recaptured       0.00      0.00      0.00       1.0
             renewal       0.00      0.00      0.00       0.0
           terminate       0.00      0.00      0.00       1.0
terminate_this_lease       0.00      0.00      0.00       0.0

            accuracy                           0.00       2.0
           macro avg       0.00      0.00      0.00       2.0
        weighted avg       0.00      0.00      0.00       2.0

Predicted Insight: legal_action


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
trained_model = "trained_model.joblib"  # Replace with your trained model's file
vectorizer_file = "vectorizer.joblib"    # Replace with your vectorizer's file

vectorizer = CountVectorizer(lowercase=True)

vectorizer = joblib.load(vectorizer_file)

classifier = joblib.load(trained_model)

new_lease_file = "test-2.txt"
with open(new_lease_file, 'r') as file:
    new_lease_text = file.read()

new_lease_features = vectorizer.transform([new_lease_text])

proba = classifier.predict_proba(new_lease_features)

insights = {}
for i, insight in enumerate(classifier.classes_):
    keyword = insight.replace("_", " ")  # Convert insight to keyword
    keyword_index = new_lease_text.lower().find(keyword)  # Find the keyword in the text
    if keyword_index != -1:
        start = new_lease_text.rfind("\n", 0, keyword_index) + 1
        end = new_lease_text.find("\n", keyword_index)
        if end == -1:
            end = len(new_lease_text)
        statement = new_lease_text[start:end].strip()
    else:
        statement = "Keyword not found"

    insights[insight] = {
        "probability": proba[0][i],
        "statement": statement
    }

# Print all predicted insights and their corresponding statements
for insight, info in insights.items():
    print(f"Insight: {insight}, Probability: {info['probability']:.4f}")
    print(f"Statement: {info['statement']}\n")

Insight: improper_conduct, Probability: 0.0000
Statement: Keyword not found

Insight: legal_action, Probability: 0.0000
Statement: Keyword not found

Insight: recapture, Probability: 0.0000
Statement: rentaT"payments due and owing shall be reduced to fully recapture this Commission Credit. The reduction in

Insight: renewal, Probability: 0.0000
Statement: continuing for a period of 10 years, with a firm term of five (5) years subject to termination and renewal

Insight: tax, Probability: 0.0000
Statement: Tax ID# 20-2579947

Insight: tax_adjustment, Probability: 0.0000
Statement: 12. This lease is subject to real estate tax adjustment as established in Paragraph 4.2 Tax Adjustment of

Insight: terminate_this_lease, Probability: 1.0000
Statement: 4. The Government may terminate this lease in whole or in part at any time on or after the fifth (5th) year of

